In [1]:
# Own Packages
from Masterarbeit_utils.model_utils import get_tokenizer, load_and_modify_model, load_pretrained_Tokenizer

# Site-Packages
import dask.dataframe as dd
import torch
import psutil
import os
import sys
import pickle as pk
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, OPTForCausalLM
from tokenizers.processors import TemplateProcessing
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from torch.utils.data import Dataset
sys.version, sys.executable

2023-07-24 16:33:35.339058: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-24 16:33:35.445359: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-24 16:33:36.033717: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


('3.10.0 (default, Jul 12 2023, 08:49:30) [GCC 12.2.0]',
 '/home/worker/.pyenv/versions/3.10.0/bin/python3.10')

In [2]:
choices = ['calculate all', 'ask for userinput', 'just calculate needed']
calculation_profile =  choices[2]
calculation_profile

'just calculate needed'

In [3]:
"""
The Paths to important folders have to be changed for your system.
"""

# Name of this experiment
model_name = 'gal_125_1'

# This folder will be created and filled with txt.files for each sample after you run the Pytorch Dataset Notebook
dataset_folder = f'data/dataset_samples'

# The folder at which the model will be saved. This folder has to be created for your system 
model_folder = f'data/models/{model_name}'
os.makedirs(model_folder, exist_ok=True)


# Folder in which the tokenizer will be saved
tokenizer_folder = f'data/tokenizers/{model_name}'
os.makedirs(tokenizer_folder, exist_ok=True)

# Folder at which all pickle files are stored. This folder is fixed for this project and should not be changed
dump_dir = r'PK_DUMP'

# Model parameters 
'''
mini	125 M
base	1.3 B
standard	6.7 B
large	30 B
huge	120 B'''
base_model_name = 'mini'

# All new Torch-objects will be by default in this dtype
# if default_type = float16 fp16 must be False
default_dtype = torch.bfloat16
torch.backends.cuda.matmul.allow_tf32 = True
torch.set_default_dtype(default_dtype)

# Default device on which the model will be loaded
default_device = 'cpu'

# Number of GPUs the model will be parallelised to 
num_gpus = 1
# If you change 'default_device' to 'cpu', make sure to set num_gpus to zero.
if default_device == 'cpu':
    num_gpus = 0

tensor_parallel = False


# Creating the Tokenizer

In [4]:
if calculation_profile == choices[0]:
    i = 'y'
elif calculation_profile == choices[1]:  
    i = input("This creates a new tokenizer instance and saves it, if you want to proceed write y: ")
else:
    i = 'n'

if i != 'y' and os.path.isfile(f'{tokenizer_folder}/tokenizer.json'):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_folder)
    n_f_terms = len(tokenizer) - tokenizer.vocab_size
    print('Loadede Tokenizer from serialized instance!')    
    print(f'There are {n_f_terms} different F-Terms in the whole Dataset!')
    
else:
    # Loads a pretrained Tokenizer for the galactica model and adds an additional token for each F-Term
    tokenizer = get_tokenizer(dump_dir)
    
    # The Tokenizer contained initially 50000 Tokens which are stored as the vocab-size.
    # The vocab_size attribute is not updated when the additional tokens are added to the tokenizer
    n_f_terms = len(tokenizer) - tokenizer.vocab_size
    tokenizer.save_pretrained(tokenizer_folder)
    print(f'There are {n_f_terms} different F-Terms in the whole Dataset!')

Loadede Tokenizer from serialized instance!
There are 378165 different F-Terms in the whole Dataset!


# Loading The Model

In [5]:
model = OPTForCausalLM.from_pretrained(f'{model_folder}/checkpoint-140000', torch_dtype=default_dtype, low_cpu_mem_usage=True,
                                           device_map=None, max_memory={'cpu': psutil.virtual_memory().available})

In [6]:
class JapPatDataset(Dataset):
    """Dataset containing Japanese patents and their F-Term classification"""
    def __init__(self, data_folder, tokenizer):
        """
        data_folder: path to folder containing the text samples
        tokenizer: tokenizer instance with added additional Tokens for F-Terms
        """
        super(Dataset).__init__()
        self.data_folder = data_folder
        # This has to be manually set to the ammount of files in the 'dataset_samples' folder. Calculating the number of files in this folder would take forever.
        # A to low number would lead to samples missing from the dataset.
        # A to high number would raise a FileNotFound error.
        self.l = len(os.listdir(data_folder))
        #self.l = 10000
        self.tokenizer = tokenizer
        
    def __len__(self):
        return self.l
    
    def __getitem__(self, idx):
        try:
            with open(f'{self.data_folder}/{idx}.txt', 'r', encoding='utf-8') as f:
                item = f.read()
        except FileNotFoundError:
            raise FileNotFoundError
        
        # Tokenizing the item 
        # The Tokenizer will return a dict with the encoded text as 'input_ids', 
        # a mask which shows the tokens types this will not be needed for our applications
        # and a mask for the attention mechanism as 'attention_mask' The attention mask will be needed to indicate, that the 
        # model should not attend to <pad> tokens.
        
        output = self.tokenizer(item)  
        output.pop('token_type_ids')
        return output

In [7]:
train_dataset = JapPatDataset(f'{dataset_folder}/train', tokenizer)
validation_dataset = JapPatDataset(f'{dataset_folder}/validation', tokenizer)

In [8]:
def classic_accurracy(batch, model, top_k=1):
    """
    Classic prediction accuracy metric. 
    This function should be applied to a batch of samples,
    which were tokenized by a tokenizer instance.
    This function returns the procentual accuracy metric as well as the total number of correct predictions
    and the total number of predictions in this batch

    :batch: batch of samples from validation dataset
    :model: model which should be testet
    :top_k: top k predictions which should be investigated for a correct result.
    """
    with torch.no_grad():
        model.eval()
        logits = model(**batch, output_hidden_states=False, return_dict=True)['logits']

    input_ids = batch['input_ids']
    labels = input_ids[:, 1:]
    print(input_ids.shape, labels.shape)
    print(logits)
    

class Batch_DataLoader():
    """
    This class converts a dataset to a iterable dataloader, which loads padded patches of data.   
    """
    def __init__(self,
                 dataset, 
                 batchsize=10,
                 datacollator=DataCollatorWithPadding(tokenizer, return_tensors='pt')):

        self.dataset = dataset
        self.batchsize = batchsize
        self.l = len(dataset)//batchsize + 1
        self.datacollator = datacollator
        self.current = 0

    def __len__(self):
        return self.l

    def __iter__(self):
        self.current = 0
        return self

    def __next__(self):
        batch = [self.dataset[i] for i in range(self.current, self.current+self.batchsize)]
        batch = self.datacollator(batch)
        self.current += self.batchsize
        return batch

In [9]:
validation_loader = Batch_DataLoader(validation_dataset, 100)

for batch in validation_loader:
    classic_accurracy(batch, model)
    break

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


torch.Size([100, 319]) torch.Size([100, 318])
tensor([[[ 1.3770e-01,  1.7266e+00, -3.2031e-01,  ..., -1.8047e+00,
          -2.6406e+00, -2.3750e+00],
         [ 9.7500e+00, -2.6719e+00, -2.0625e+00,  ...,  6.2109e-01,
           5.7031e-01,  8.5547e-01],
         [ 9.8750e+00, -2.2969e+00, -1.6875e+00,  ...,  7.7734e-01,
           8.1250e-01,  8.5156e-01],
         ...,
         [-1.4844e+00,  1.2969e+00,  1.3428e-02,  ..., -1.2266e+00,
          -2.5938e+00, -2.6250e+00],
         [-1.4844e+00,  1.2969e+00,  1.3428e-02,  ..., -1.2266e+00,
          -2.5938e+00, -2.6250e+00],
         [-1.4844e+00,  1.2969e+00,  1.3428e-02,  ..., -1.2266e+00,
          -2.5938e+00, -2.6250e+00]],

        [[ 1.3770e-01,  1.7266e+00, -3.2031e-01,  ..., -1.8047e+00,
          -2.6406e+00, -2.3750e+00],
         [ 9.7500e+00, -2.6719e+00, -2.0625e+00,  ...,  6.2109e-01,
           5.7031e-01,  8.5547e-01],
         [ 9.8750e+00, -2.2969e+00, -1.6875e+00,  ...,  7.7734e-01,
           8.1250e-01,  8.5156

In [ ]:
with open(f'{dump_dir}/full_descriptions.pk', 'rb') as f:
    full_descriptions_dict = pk.load(f)

In [ ]:
test_sample = validation_dataset[1]
test_sample_text = tokenizer.decode(test_sample['input_ids'])
test_sample_text


In [ ]:
x = 4
input_ids=torch.tensor([test_sample['input_ids'][:-x]])
attention_mask=torch.tensor([test_sample['attention_mask'][:-x]])
print(input_ids.shape, input_ids)
print('vocab_size', model.config.vocab_size)
output = model(input_ids, attention_mask, output_attentions=False, output_hidden_states=False, return_dict=True)
logits = output['logits']
indices = torch.max(logits, dim=-1)
indices = indices[1]
indices += 50000
indices, tokenizer.decode(test_sample['input_ids'][-x]), full_descriptions_dict[ tokenizer.decode(test_sample['input_ids'][-x])[:-1]]

In [ ]:
tokenizer.decode(indices[0])

In [ ]:
pred_token = tokenizer.decode(indices[0][-1])
targ_token = tokenizer.decode(test_sample['input_ids'][-x])

print('Predicted Token: ', pred_token, 'Target Token: ', targ_token)

In [ ]:
print('Abstract: ')
print(tokenizer.decode(test_sample['input_ids'][:-x]))
print(' ')

print('Predicted Description: ')
print(full_descriptions_dict[pred_token[:-1]])
print(' ')
print('Target Description: ')
print(full_descriptions_dict[targ_token[:-1]])

In [ ]:
s=torch.nn.Softmax()
r = s(logits)
r

In [ ]:
logits

In [ ]:
logits.shape